In [ ]:
# 종속 패키지(dependencies) 설치
!pip install -U torch torchvision cython
!pip install -U 'git+https://github.com/facebookresearch/fvcore.git' 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
torch.__version__

!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

In [ ]:
# 인스톨이 완료되면 Colab Runtime을 다시 시작해주세요. 

# 기본 설정
import os
import numpy as np
import json
from detectron2.structures import BoxMode
import itertools

# detectron2 logger 설정
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# 자주 사용하는 라이브러리 임폴트
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# 자주 사용하는 detectron2 유틸 임폴트 
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [ ]:
!git clone https://github.com/sosobam/AI_Jump_up.git

In [ ]:
# detectron2에서 데이터셋 정보를 로드하는 형식에 맞춰 함수를 작성합니다.
def get_PTN_dicts(img_dir):
    json_file = os.path.join(img_dir, "annotations.json")
    with open(json_file) as f:
        imgs_anns = json.load(f)

    dataset_dicts = []
    
    idx = 0
    #for idx, v in enumerate(imgs_anns.values()):
    for key, val in imgs_anns.items():
        record = {}
        
        #filename = os.path.join(img_dir, v["filename"])
        filename = os.path.join(img_dir, key)
        height, width = cv2.imread(filename).shape[:2]
        
        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width
      
        #annos = v["regions"]
        annos = val["instances"]
        objs = []
        #for _, anno in annos.items():
        for anno in annos:
            #assert not anno["region_attributes"]
            #anno = anno["shape_attributes"]
            #px = anno["all_points_x"]
            #py = anno["all_points_y"]
            points = anno["points"]
            px = []
            py = []

            for i in range(0,int(len(points)/2)):
              px = px + [points[i*2]]
              py = px + [points[i*2+1]]

            poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
            poly = list(itertools.chain.from_iterable(poly))

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": 0,
                "iscrowd": 0
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog
for d in ["train", "val"]:
    DatasetCatalog.register("PTN_" + d, lambda d=d: get_PTN_dicts(d))
    MetadataCatalog.get("PTN_" + d).set(thing_classes=["PTN"])
balloon_metadata = MetadataCatalog.get("PTN_train")

In [ ]:
# 데이터가 제대로 로드되었는지 확인
dataset_dicts = get_PTN_dicts("AI_Jump_up/TRAIN")
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=balloon_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])